In [ ]:
import time
from contextlib import contextmanager
import pandas as pd
import numpy as np
import gc
import csv
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


class DataGeneration:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __get_columns(self, reader):
        for row in reader:
            columns = row
            break
        return columns

    def __get_data_from_idx(self, csv_path, idx, col_id, idx_list):
        column_types_df = pd.read_csv(csv_path+'.csv', nrows=1)
        file = open(csv_path+idx+'.csv')
        reader = csv.reader(file)
        selected_data = pd.DataFrame(columns=self.__get_columns(reader))
        selected_data = selected_data.astype(dtype=column_types_df.dtypes)
        for row in reader:
            if row[col_id] in idx_list:
                line = []
                for item in row:
                    if item == '':
                        line.append(float(np.nan))
                    else:
                        try:
                            line.append(int(item))
                        except:
                            try:
                                line.append(float(item))
                            except:
                                line.append(str(item))
                selected_data.loc[len(selected_data)] = line
        return selected_data

    def __one_hot_encoder(self, df, nan_as_category=True):  # One-hot encoding for categorical columns with get_dummies
        original_columns = list(df.columns)
        categorical_columns = [
            col for col in df.columns if df[col].dtype == 'object']
        df = pd.get_dummies(df, columns=categorical_columns,
                            dummy_na=nan_as_category)
        new_columns = [c for c in df.columns if c not in original_columns]
        return df, new_columns

    def __application_train(self, df, nan_as_category=False):
        # Optional: Remove 4 applications with XNA CODE_GENDER (train set)
        df = df[df['CODE_GENDER'] != 'XNA']
        # Categorical features with Binary encode (0 or 1; two categories)
        for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
            df[bin_feature], uniques = pd.factorize(df[bin_feature])
        # Categorical features with One-Hot encode
        df, cat_cols = self.__one_hot_encoder(df, nan_as_category)
        # Transform hot encoded categorical columns type as 'category' type
        df[cat_cols] = df[cat_cols].astype("category")
        # NaN values for DAYS_EMPLOYED: 365.243 -> nan
        if 365243 in df['DAYS_EMPLOYED']:
            df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
        # Some simple new features (percentages)
        df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
        df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
        df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
        df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
        df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
        gc.collect()
        return df

    def __bureau_and_balance(self, bureau, bb, nan_as_category=True):
        bureau, bureau_cat = self.__one_hot_encoder(bureau, nan_as_category)
        bb, bb_cat = self.__one_hot_encoder(bb, nan_as_category)
        # Bureau balance: Perform aggregations and merge with bureau.csv
        bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
        for col in bb_cat:
            bb_aggregations[col] = ['mean']
        bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
        bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper()
                                  for e in bb_agg.columns.tolist()])
        bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
        bureau.drop(['SK_ID_BUREAU'], axis=1, inplace=True)
        del bb, bb_agg
        gc.collect()

        # Bureau and bureau_balance numeric features
        num_aggregations = {
            'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
            'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
            'DAYS_CREDIT_UPDATE': ['mean'],
            'CREDIT_DAY_OVERDUE': ['max', 'mean'],
            'AMT_CREDIT_MAX_OVERDUE': ['mean'],
            'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
            'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
            'AMT_CREDIT_SUM_OVERDUE': ['mean'],
            'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
            'AMT_ANNUITY': ['max', 'mean'],
            'CNT_CREDIT_PROLONG': ['sum'],
            'MONTHS_BALANCE_MIN': ['min'],
            'MONTHS_BALANCE_MAX': ['max'],
            'MONTHS_BALANCE_SIZE': ['mean', 'sum']
        }
        # Bureau and bureau_balance categorical features
        cat_aggregations = {}
        for cat in bureau_cat:
            cat_aggregations[cat] = ['mean']
        for cat in bb_cat:
            cat_aggregations[cat + "_MEAN"] = ['mean']
        bureau_agg = bureau.groupby('SK_ID_CURR').agg(
            {**num_aggregations, **cat_aggregations})
        bureau_agg.columns = pd.Index(
            ['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
        # Bureau: Active credits - using only numerical aggregations
        if 'CREDIT_ACTIVE_Active' in bureau.columns:
            active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
            active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
            active_agg.columns = pd.Index(
                ['ACTIVE_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
            bureau_agg = bureau_agg.join(
                active_agg, how='left', on='SK_ID_CURR')
            del active, active_agg
            gc.collect()
        # Bureau: Closed credits - using only numerical aggregations
        if 'CREDIT_ACTIVE_Closed' in bureau.columns:
            closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
            closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
            closed_agg.columns = pd.Index(
                ['CLOSED_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
            bureau_agg = bureau_agg.join(
                closed_agg, how='left', on='SK_ID_CURR')
            del closed, closed_agg
            gc.collect()
        del bureau
        gc.collect()
        return bureau_agg

    def __previous_applications(self, prev, nan_as_category=True):
        prev, cat_cols = self.__one_hot_encoder(prev, nan_as_category)
        # Days 365.243 values -> nan
        prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
        prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
        prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
        prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
        prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)
        # Add feature: value ask / value received percentage
        prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
        # Previous applications numeric features
        num_aggregations = {
            'AMT_ANNUITY': ['min', 'max', 'mean'],
            'AMT_APPLICATION': ['min', 'max', 'mean'],
            'AMT_CREDIT': ['min', 'max', 'mean'],
            'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
            'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
            'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
            'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
            'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
            'DAYS_DECISION': ['min', 'max', 'mean'],
            'CNT_PAYMENT': ['mean', 'sum'],
        }
        # Previous applications categorical features
        cat_aggregations = {}
        for cat in cat_cols:
            cat_aggregations[cat] = ['mean']
        prev_agg = prev.groupby('SK_ID_CURR').agg(
            {**num_aggregations, **cat_aggregations})
        prev_agg.columns = pd.Index(
            ['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
        # Previous Applications: Approved Applications - only numerical features
        if 'NAME_CONTRACT_STATUS_Approved' in prev.columns:
            approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
            approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
            approved_agg.columns = pd.Index(
                ['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
            prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
            del approved, approved_agg
        # Previous Applications: Refused Applications - only numerical features
        if 'NAME_CONTRACT_STATUS_Refused' in prev.columns:
            refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
            refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
            refused_agg.columns = pd.Index(
                ['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
            prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
            del refused, refused_agg
        del prev
        gc.collect()
        return prev_agg

    def __pos_cash(self, pos, num_rows=None, nan_as_category=True):
        pos, cat_cols = self.__one_hot_encoder(pos, nan_as_category)
        # Features
        aggregations = {
            'MONTHS_BALANCE': ['max', 'mean', 'size'],
            'SK_DPD': ['max', 'mean'],
            'SK_DPD_DEF': ['max', 'mean']
        }
        for cat in cat_cols:
            aggregations[cat] = ['mean']

        pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
        pos_agg.columns = pd.Index(
            ['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
        # Count pos cash accounts
        pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
        del pos
        gc.collect()
        return pos_agg

    def __installments_payments(self, ins, num_rows=None, nan_as_category=True):
        ins, cat_cols = self.__one_hot_encoder(ins, nan_as_category)
        # Percentage and difference paid in each installment (amount paid and installment value)
        ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
        ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
        # Days past due and days before due (no negative values)
        ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
        ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
        ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
        ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
        # Features: Perform aggregations
        aggregations = {
            'NUM_INSTALMENT_VERSION': ['nunique'],
            'DPD': ['max', 'mean', 'sum'],
            'DBD': ['max', 'mean', 'sum'],
            'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
            'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
            'AMT_INSTALMENT': ['max', 'mean', 'sum'],
            'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
            'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
        }
        for cat in cat_cols:
            aggregations[cat] = ['mean']
        ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
        ins_agg.columns = pd.Index(
            ['INSTAL_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
        # Count installments accounts
        ins_agg['INSTAL_COUNT'] = ins.groupby('SK_ID_CURR').size()
        del ins
        gc.collect()
        return ins_agg

    def __credit_card_balance(self, cc, nan_as_category=True):
        cc, cat_cols = self.__one_hot_encoder(cc, nan_as_category)
        # General aggregations
        cc.drop(['SK_ID_PREV'], axis=1, inplace=True)
        cc_agg = cc.groupby('SK_ID_CURR').agg(
            ['min', 'max', 'mean', 'sum', 'var'])
        cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper()
                                  for e in cc_agg.columns.tolist()])
        # Count credit card lines
        cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
        del cc
        gc.collect()
        return cc_agg

    def process_database(self, dataframes):
        df = self.__application_train(dataframes['application_train'])
        bureau = self.__bureau_and_balance(dataframes['bureau'], dataframes['bureau_balance'])
        print("Bureau df shape:", bureau.shape)
        df = df.join(bureau, how='left', on='SK_ID_CURR')
        prev = self.__previous_applications(
            dataframes['previous_application'])
        print("Previous applications df shape:", prev.shape)
        df = df.join(prev, how='left', on='SK_ID_CURR')
        pos = self.__pos_cash(dataframes['POS_CASH_balance'])
        print("Pos-cash balance df shape:", pos.shape)
        df = df.join(pos, how='left', on='SK_ID_CURR')
        ins = self.__installments_payments(dataframes['installments_payments'])
        print("Installments payments df shape:", ins.shape)
        df = df.join(ins, how='left', on='SK_ID_CURR')
        cc = self.__credit_card_balance(dataframes['credit_card_balance'])
        print("Credit card balance df shape:", cc.shape)
        df = df.join(cc, how='left', on='SK_ID_CURR')

        # Replace infinity values by nan
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df = df.rename(columns=lambda x: re.sub('[^A-Za-z0-9_]+', '', x))
        return df

    def write_idx_data(self, files_dict, curr_id):
        for key, value in files_dict.items():
            df = self.__get_data_from_idx(
                'data/'+value[0], '', value[1], [curr_id])
            df.to_csv('data/'+value[0]+curr_id+'.csv',
                      index_label='SK_ID_CURR', index=False)

            if value[0] == 'bureau':
                bb = self.__get_data_from_idx(
                    'data/bureau_balance', '', 0, list(df['SK_ID_BUREAU']))
                bb.to_csv('data/bureau_balance'+curr_id+'.csv',
                          index_label='SK_ID_BUREAU', index=False)
        return

    def get_idx_data(self, files_dict, curr_id):
        dataframes = {}
        for key, value in files_dict.items():
            dataframes[value[0]] = self.__get_data_from_idx(
                'data/'+value[0], curr_id, value[1], [curr_id])
            
            if value[0] == 'bureau':
                dataframes['bureau_balance'] = self.__get_data_from_idx(
                    'data/bureau_balance', '', 0, list(dataframes[value[0]]['SK_ID_BUREAU']))
                
        return dataframes

In [ ]:
obj = DataGeneration()

files_dict = {
    1: ('application_train', 0),
    2: ('bureau', 0),
    3: ('previous_application', 1),
    4: ('POS_CASH_balance', 1),
    5: ('installments_payments', 1),
    6: ('credit_card_balance', 1)
}

idx_curr = '100003'
obj.write_idx_data(files_dict, idx_curr)
dataframes = obj.get_idx_data(files_dict, idx_curr)
df = obj.process_database(dataframes)
print(df)

In [ ]:
import csv
import pandas as pd

file_list = [
    'data/application_train.csv',
    'data/credit_card_balance.csv',
    'data/bureau_balance.csv',
    'data/POS_CASH_balance.csv',
    'data/bureau.csv',
    'data/installments_payments.csv',
    'data/previous_application.csv'
]

unique_items_df = pd.DataFrame()
for file in file_list:
    dataframe = pd.read_csv(file)
    df_obj = dataframe.select_dtypes(include='object')
    for column in df_obj.columns:
        if column not in unique_items_df.columns:
            unique_items_df = pd.concat([unique_items_df, pd.DataFrame(
                df_obj[column][df_obj[column].notnull()].unique(), columns=[column])], axis=1)
        else:
            list1 = list(unique_items_df[column])
            list2 = list(df_obj[column].unique())
            list1.extend(list2)
            new_set = set(list1)
            unique_items_df.drop(column, inplace=True, axis=1)
            unique_items_df = pd.concat([unique_items_df, pd.DataFrame(
                df_obj[column].unique(), columns=[column])], axis=1) 
            
unique_items_df.loc[-1] = 'undefined'
unique_items_df.index = unique_items_df.index + 1  # shifting index
unique_items_df.sort_index(inplace=True)

unique_items_df.to_csv("data/unique_items_df.csv", index=False)

In [ ]:
print(unique_items_df.columns)
unique_items_df.loc[-1] = 'undefined'
unique_items_df.index = unique_items_df.index + 1  # shifting index
unique_items_df.sort_index(inplace=True) 

In [ ]:
unique_items_df.to_csv("data/unique_items_df.csv", index=False)

In [ ]:
import pandas as pd

data=pd.read_csv('data/application_train.csv', usecols=['SK_ID_CURR']).T.values.tolist()[0]

In [ ]:
data